# Training the Neural Networks

The pretrained neural networks are available as part of the download from notebook [2. Dataset Generation](2.%20Dataset%20Generation.ipynb). If you haven't downloaded the models from the release yet, we suggest that you do now.

In case you want to train your own models, here is the code to do so.

In [1]:
# Required on some Windows machines
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH "] = "true"

import arrow
import tensorflow as tf
from tqdm.notebook import tqdm

from deepalign import Dataset
from deepalign import fs
from deepalign.alignments import ConfNet

We can get all dataset filenames using this helper method.

In [2]:
datasets = sorted([f.name for f in fs.get_event_log_files()])

Now we can train a ConfNet model for each of the datasets using the following for loop. It will create a version of ConfNet with no attributes `(0, 0)`, only case attributes `(0, 1)`, only event attributes `(1, 0)`, and both `(1, 1)`.

In [3]:
for dataset_name in datasets:
    for ea, ca in [(0, 0), (0, 1), (1, 0), (1, 1)]:
        start_time = arrow.now()
        dataset = Dataset(dataset_name, use_case_attributes=ca, use_event_attributes=ea)
        if ca and dataset.num_case_attributes == 0:
            continue
        confnet = ConfNet(dataset, use_case_attributes=ca, use_event_attributes=ea)
        confnet.fit(dataset, batch_size=100, epochs=50, validation_split=0.1,
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)])
        confnet.save(
            str(fs.MODEL_DIR / f'{dataset_name}_{confnet.identifier}_{start_time.format(fs.DATE_FORMAT)}'))

Epoch 1/50
45/45 [==============================] - 1s 19ms/step - loss: 1.8948 - val_loss: 0.9985
Epoch 2/50
45/45 [==============================] - 0s 9ms/step - loss: 0.6449 - val_loss: 0.3673

# Creating the Baseline Models

In [4]:
from deepalign.alignments.processmining import OptimalCostAligner
from deepalign.alignments.processmining import HeuristicsMinerAligner
from deepalign.alignments.processmining import InductiveMinerAligner

In [5]:
datasets = sorted([f.name for f in fs.get_event_log_files()])

In [6]:
aligners = [OptimalCostAligner, HeuristicsMinerAligner, InductiveMinerAligner]

In [7]:
for aligner_class in tqdm(aligners):
    for dataset_name in tqdm(datasets):
        dataset = Dataset(dataset_name)
        aligner = aligner_class()
        aligner.fit(dataset)
        file_name = f'{dataset_name}_{aligner.abbreviation}'
        aligner.save(file_name)

100%|██████████| 5000/5000 [00:00<00:00, 6784.26it/s]
Y:\Repos_Uni\deepalign\deepalign\processmining\log.py:58: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(self.cases)[indices]
